In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 15.6 MB/s 


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
from torchmetrics import Accuracy
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.transforms import Resize
from torchvision.transforms import Compose

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class Inception(nn.Module):
  def __init__(self, c1, c2, c3, c4, **kwargs):
    super(Inception, self).__init__(**kwargs)
    self.b1_1 = nn.LazyConv2d(c1, kernel_size = 1)
    self.b2_1 = nn.LazyConv2d(c2[0], kernel_size = 1)
    self.b2_2 = nn.LazyConv2d(c2[1], kernel_size = 3, padding = 1)
    self.b3_1 = nn.LazyConv2d(c3[0], kernel_size = 1)
    self.b3_2 = nn.LazyConv2d(c3[1], kernel_size = 5, padding = 2)
    self.b4_1 = nn.MaxPool2d(kernel_size = 3, stride = 1, padding = 1)
    self.b4_2 = nn.LazyConv2d(c4, kernel_size = 1)

  def forward(self, x):
    b1 = F.relu(self.b1_1(x))
    b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
    b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
    b4 = F.relu(self.b4_2(self.b4_1(x)))
    return torch.cat((b1, b2, b3, b4), dim = 1)

In [ ]:
class GoogleNet(nn.Module):
  def __init__(self, lr = 0.1, num_classes = 10):
    super(GoogleNet, self).__init__()
    self.lr = lr
    self.num_classes = num_classes
    self.net = nn.Sequential(self.b1(), self.b2(), self.b3(), self.b4(), self.b5(), nn.LazyLinear(num_classes))

  def b1(self):
    return nn.Sequential(
        nn.LazyConv2d(64, kernel_size = 7, stride = 2, padding = 3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 3, stride =2, padding = 1)
    )
  
  def b2(self):
    return nn.Sequential(
        nn.LazyConv2d(64, kernel_size = 1),
        nn.ReLU(),
        nn.LazyConv2d(192, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
    )

  def b3(self):
    return nn.Sequential(
        Inception(64, (96, 128), (16,32), 32),
        Inception(128, (128, 192), (32, 96), 64),
        nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
    )

  def b4(self):
    return nn.Sequential(
        Inception(192, (96, 208), (16, 48), 64),
        Inception(160, (112, 224), (42, 64), 64),
        Inception(128, (128, 256), (24, 64), 64),
        Inception(112, (144, 288), (42, 64), 64),
        Inception(256, (160, 320), (32, 128), 128),
        nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
    )
  
  def b5(self):
    return nn.Sequential(
        Inception(256, (160, 320), (42, 128), 128),
        Inception(384, (192, 384), (48, 128), 128),
        nn.AdaptiveAvgPool2d((1,1)),
        nn.Flatten()
    )

In [ ]:
transform = Compose(
    [Resize((96, 96)),
    ToTensor()]
)

In [ ]:
train_data = FashionMNIST(root = './data', train = True,
                          transform = transform, target_transform = None,
                          download = True)

test_data = FashionMNIST(root = './data', train = False,
                          transform = transform, target_transform = None,
                          download = True)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
torch.manual_seed(42)

train_loader = DataLoader(train_data, batch_size = 128, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 128, shuffle = True)

In [ ]:
model = GoogleNet(lr = 0.01).to(device)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
acc = Accuracy(task = 'multiclass', num_classes = 10).to(device)

In [ ]:
def train_step(model, train_loader, criterion, optimizer, acc, device):
  train_loss = 0
  train_acc = 0

  for X, y in train_loader:
    X, y = X.to(device), y.to(device)
    preds = model(X)
    loss = criterion(preds, y)
    train_loss += loss.item()
    train_acc += acc(preds, y).item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss /= len(train_loader)
  train_acc /= len(train_loader)

  return train_loss, train_acc

In [ ]:
def test_step(model, test_loader, criterion, acc, device):
  test_loss = 0
  test_acc = 0

  model.eval()
  with torch.inference_mode():
    for X, y in test_loader:
      X, y = X.to(device), y.to(device)
      preds = model(X)
      test_loss += criterion(preds, y).item()
      test_acc += acc(preds, y).item()

    test_loss /= len(test_loader)
    test_acc /= len(test_loader)

    return test_loss, test_acc

In [ ]:
class ConvBlock(nn.Module):
  def __init__(self, in_fts, out_fts, k, s, p):
    super(ConvBlock, self).__init__()
    self.convolution = nn.Sequential(
        nn.Conv2d(in_fts, out_fts, kernel_size = k, stride = s, padding = p),
        nn.ReLU()
    )

    def forward(self, x):
      x = self.convolution(x)
      return x

In [ ]:
class ReduceConvBlock(nn.Module):
  def __init__(self, in_fts, out_fts_1, out_fts_2, k, p):
    super(ReduceConvBlock, self).__init__()
    self.redConv = nn.Sequential(
        nn.Conv2d(in_fts, out_fts_1, kernel_size = 1, stride = 1),
        nn.ReLU(),
        nn.Conv2d(out_fts_1, out_fts_2, kernel_size = k, stride = 1),
        nn.ReLU()
    )

  def forward(self, input_img):
    x = self.redConv(input_img)
    return x

In [ ]:
class AuxClassifier(nn.Module):
  def __init__(self, in_fts, num_classes = 10):
    super(AuxClassifier, self).__init__()
    self.avgpool = nn.AvgPool2d(kernel_size = 5, stride = 3)
    self.conv = nn.Conv2d(in_fts, 128, kernel_size = 1, stride = 1)
    self.relu = nn.ReLU()
    self.fc = nn.Linear(4 * 4 * 128, 1024)
    self.dropout = nn.Dropout(p = 0.7)
    self.classifier = nn.Linear(1024, num_classes)

  def forward(self, input_img):
    N = input_img.shape[0]
    x = self.avgpool(input_img)
    x = self.conv(x)
    x = self.relu(x)
    x = x.reshape(N, -1)
    x = self.fc(x)
    x = self.dropout(x)
    x = self.classifier(x)
    return x

In [ ]:
class InceptionModule(nn.Module):
  def __init__(self, curr_in_fts, f_1x1, f_3x3_r, f_3x3, f_5x5_r, f_5x5, f_pool_proj):
    super(InceptionModule, self).__init__()
    self.conv1 = ConvBlock(curr_in_fts, f_1x1, 1, 1, 0)
    self.conv2 = ReduceConvBlock(curr_in_fts, f_3x3_r, f_3x3, 3, 1)
    self.conv3 = ReduceConvBlock(curr_in_fts, f_5x5_r, f_5x5, 5, 2)

    self.pool_proj = nn.Sequential(
        nn.MaxPool2d(kernel_size = 1, stride = 1),
        nn.Conv2d(curr_in_fts, f_pool_proj, kernel_size = 1, stride = 1),
        nn.ReLU()
    )
  
  def forward(self, input_img):
    out1 = self.conv1(input_img)
    out2 = self.conv2(input_img)
    out3 = self.conv3(input_img)
    out4 = self.pool_proj(input_img)

    x = torch.cat([out1, out2, out3, out4], dim = 1)

    return x

In [ ]:
class MyGoogleNet(nn.Module):
  def __init__(self, in_fts = 3, num_classes = 10):
    super(MyGoogleNet, self).__init__()
    self.conv1 = ConvBlock(in_fts, 64, 7, 2, 3)
    self.maxpool1 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)

    self.conv2 = nn.Sequential(
        ConvBlock(64, 64, 1, 1, 0),
        ConvBlock(64, 192, 3, 1, 1)
    )

    self.inception_3a = InceptionModule(192, 64, 96, 128, 16)
    self.inception_3b = InceptionModule(256, 128, 128, 192, 32, 96, 64)
    self.inception_4a = InceptionModule(480, 192, 96, 208, 16, 48, 64)
    self.inception_4b = InceptionModule(512, 160, 112, 224, 24, 64, 64)
    self.inception_4c = InceptionModule(512, 128, 128, 256, 24, 64, 64)
    self.inception_4d = InceptionModule(512, 112, 144, 288, 32, 64, 64)
    self.inception_4e = InceptionModule(528, 256, 160, 320, 32, 128, 128)
    self.inception_5a = InceptionModule(832, 256, 160, 320, 32, 128, 128)
    self.inception_5b = InceptionModule(832, 384, 192, 384, 48, 128, 128)

    self.aux_classifier1 = AuxClassifier(512, num_classes)
    self.aux_classifeir2 = AuxClassifier(528, num_classes)
    self.avgpool = nn.AdaptivePool2d(output_size = 7)
    self.classifier = nn.Sequential(
        nn.Dropout(p = 0.4),
        nn.Linear(1024 * 7 * 7, num_classes)
    )

    def forward(self, input_img):
      N = input_img.shape[0]
      x = self.conv1(input_img)
      x = self.maxpool1(x)
      x = self.conv2(x)
      x = self.maxpool1(x)
      x = self.inception_3a(x)
      x = self.inception_3b(x)
      x = self.maxpool1(x)
      x = self.inception4a(x)
      out1 = self.aux_classifier1(x)
      x = self.inception_4b(x)
      x = self.inception_4c(x)
      x = self.inception_4d(x)
      out2 = self.aux_classifier2(x)
      x = self.inception_4e(x)
      x = self.maxpool1(x)
      x = self.inception_5a(x)
      x = self.inception_5b(x)
      x = self.avgpool(x)
      x = x.reshape(N, - 1)
      x = self.classifier(x)
      if self.training == True:
        return [x, out1, out2]
      else:
        return x